<a href="https://colab.research.google.com/github/pavanreddy565/PyTorch_learning/blob/main/basics/Linerar_regresssion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

In [6]:
import numpy as np
import torch

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [ ]:
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
inputs,targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [ ]:
# Weights and biases
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
w,b

(tensor([[-0.7693, -1.3281,  0.4447],
         [-0.1458,  1.8284,  0.3992]], requires_grad=True),
 tensor([-0.3031,  0.5973], requires_grad=True))

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [ ]:
def model(x):
  return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [ ]:
preds=model(inputs)
preds,targets

(tensor([[-126.3278,  129.6232],
         [-158.7282,  173.7786],
         [-219.4117,  256.0728],
         [-119.4323,   79.1188],
         [-149.7595,  194.0082]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

# Loss Function
Before we imporve our model, we need w way to evaluat how well our model is performing. We can compare the model's predictions with the actual targets using the following method:
* calculate the difference between the two matrices(```preds``` and ```targets```)
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **Mean Squared Error**(MSE).

In [ ]:
# MSE loss
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff*diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [ ]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(32052.4785, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

# Compute gradients
with PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t to the weights and biases because they have ```requires_grad``` set to ```True```. We'll see how this is useful in just a moment.

In [ ]:
# Compute gradients
loss.backward()

the gradients are stored in the ```.grad``` property of the respective tensors. Note that the derviative of the loss w.r.t the weights matrix is itself a matrix with  same dimensions

In [ ]:
# Grdient for Weights
w,w.grad

(tensor([[-0.7693, -1.3281,  0.4447],
         [-0.1458,  1.8284,  0.3992]], requires_grad=True),
 tensor([[-19286.7031, -21801.1426, -13147.3477],
         [  6230.8730,   7180.5859,   4233.7646]]))

In [ ]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

# Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:
1. Generate Predictions
1. Calculate the loss
1. Compute gradients w.r.t the weights and biases
1. Adjust the weights by subracting a small quantity propotional to the gradient.
1. Reset the gradients to zero


In [ ]:
# Genrate predictions
preds = model(inputs)
preds

tensor([[ -57.6443,  107.2616],
        [ -68.4232,  144.3799],
        [-112.1702,  221.0745],
        [ -51.6048,   57.0980],
        [ -62.8747,  165.6942]], grad_fn=<AddBackward0>)

In [ ]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(13522.5684, grad_fn=<DivBackward0>)

In [ ]:
# Compute gradients
loss.backward()
w.grad,b.grad

(tensor([[-12191.6797, -14165.8398,  -8438.4551],
         [  3920.4077,   4692.5581,   2699.8198]]),
 tensor([-146.7434,   47.1016]))

In [ ]:
# Adjust weights & reset gradients
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()


we multiply the gradients with a small number(10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We watn to take a small step in the downhill of the gradient, not a gaint leap. This number is called the *learning rate* of the algorithm.

we Use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [ ]:
w,b

(tensor([[-0.2617, -0.7504,  0.7920],
         [-0.3096,  1.6379,  0.2875]], requires_grad=True),
 tensor([-0.2970,  0.5953], requires_grad=True))

In [ ]:
# Calculate loss
preds = model(inputs)
loss =mse(preds, targets)
loss

tensor(9286.4258, grad_fn=<DivBackward0>)

# Linear Regression using PyTorch built-ins
We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is common pattern in deeplearning, PyTorch provides several built-in functions and classes to make it easy to create and train models with jsut a few lines of code.
Let's begin by importing the ```torch.nn``` package from PyTorch, which contains utility classes for buliding neural networks.

In [2]:
import torch.nn as nn
import numpy as np

In [7]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples to illustrate how to work with large datasets in small batches. ( mini-batch gradient descent)

# Dataset and DataLoader
we'll create ```TensorDataset```, which allows access to rows form inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [8]:
from torch.utils.data import TensorDataset


In [9]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

In [10]:
from torch.utils.data import DataLoader

In [12]:
# Define data loader
batch_size = 5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)


In [13]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break


tensor([[91., 87., 65.],
        [68., 97., 70.],
        [73., 66., 44.],
        [92., 87., 64.],
        [91., 88., 64.]])
tensor([[ 80., 102.],
        [102., 120.],
        [ 57.,  69.],
        [ 82., 100.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

# nn.Linear
Instead of intializing the weights & baises manually, we can define the model using the ```nn.Linear``` class from PyTorch, which does it automatically.

In [14]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4952, -0.3022,  0.3981],
        [-0.0846, -0.1073,  0.4403]], requires_grad=True)
Parameter containing:
tensor([ 0.5217, -0.4699], requires_grad=True)


In [23]:
?nn.Linear #gives documentation of nn.Linear

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [15]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4952, -0.3022,  0.3981],
         [-0.0846, -0.1073,  0.4403]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5217, -0.4699], requires_grad=True)]

In [16]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-38.7562,   5.0967],
        [-45.6556,  10.5668],
        [-59.9667,   3.3273],
        [-48.2514,   2.5757],
        [-34.7908,  14.2123],
        [-38.9492,   5.1193],
        [-44.9552,  11.1145],
        [-60.0637,   3.6830],
        [-48.0584,   2.5530],
        [-33.8975,  14.7372],
        [-38.0559,   5.6443],
        [-45.8485,  10.5895],
        [-60.6670,   2.7796],
        [-49.1447,   2.0507],
        [-34.5979,  14.1896]], grad_fn=<AddmmBackward0>)

# Loss Function
instead of defining a loss function manually, we can use the bulit-in loss function ```mse_loss```

In [17]:
# Import nn.functional
import torch.nn.functional as F

In [18]:
# Define loss function
loss_fn = F.mse_loss

In [19]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(12194.5801, grad_fn=<MseLossBackward0>)


# Optimizer
Instead of manually manipulating  the model's weights & biases using gradients, we can use the optimizer ```optim.SGD```. SGD is short for "stochastic gradient descent". The term *stochastic* indicates the samples are selected in random batches instead of single group.


In [20]:
# Define Optmizer
opt = torch.optim.SGD(model.parameters(),lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

In [21]:
# Utility functions to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

  #Repeat for given number of epochs
  for epoch in range(num_epochs):

    # Train with batches of data
    for xb,yb in train_dl:

      # 1. Generate predictions
      pred=model(xb)

      # 2. Calculate loss
      loss = loss_fn(pred,yb)

      # 3. Compute Gradients
      loss.backward()

      # 4. Update parameters using gradients
      opt.step()

      # 5. Reset the gradients to zero
      opt.zero_grad

    # Print the progress
    if (epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [22]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 18733.8281
Epoch [20/100], Loss: 26395.0527
Epoch [30/100], Loss: 18207.0410
Epoch [40/100], Loss: 20112.0195
Epoch [50/100], Loss: 19066.7070
Epoch [60/100], Loss: 6272.5215
Epoch [70/100], Loss: 174.1742
Epoch [80/100], Loss: 3022.0930
Epoch [90/100], Loss: 10459.2061
Epoch [100/100], Loss: 16756.8945
